In [27]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import random
from collections import defaultdict

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from pymongo import MongoClient

# Get URLs

In [28]:
chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

In [29]:
query = 'covid-19 comedy'
youtube_search = 'https://www.youtube.com/results?search_query='
#channel_filter = '&sp=EgIQAg%253D%253D'
youtube_query = youtube_search + query.replace(' ', '+') # + channel_filter
youtube_query

'https://www.youtube.com/results?search_query=covid-19+comedy'

In [30]:
driver = webdriver.Chrome(chromedriver)
driver.get(youtube_query)

In [44]:
#scroll
for i in range(150):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1)

In [48]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
contents_div = soup.find('div', id='contents')

urls = []
for title in contents_div.find_all('a', id='thumbnail', attrs={'class': 'yt-simple-endpoint inline-block style-scope ytd-thumbnail'}):
    channel_url = f"https://www.youtube.com{title['href']}"
    urls.append(channel_url)

In [51]:
with open('urls_comedy_covid_644.pkl', 'wb') as picklefile:
    pickle.dump(urls, picklefile)

In [49]:
len(urls)

644

# Set Up DB

In [53]:
client = MongoClient()

In [61]:
client.list_database_names()

['admin', 'books', 'config', 'local', 'outings']

In [62]:
db = client['comedy_covid']

In [65]:
db.create_collection("covid_comedy_youtube_comments")

In [80]:
db.list_collection_names()

['covid_comedy_youtube_comments_test', 'covid_comedy_youtube_comments']

In [67]:
client.list_database_names()

['admin', 'books', 'comedy_covid', 'config', 'local', 'outings']

In [79]:
#covid_comedy_youtube_comments_test.insert_one({'y':1})

In [85]:
covid_comedy_youtube_comments = db.get_collection('covid_comedy_youtube_comments')

# Get Comments with API

In [87]:
# Sample Python code for youtube.comments.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python
 
import os

import googleapiclient.discovery

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = "XXXXXXXXXX"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)
    
    for url in urls:
        try:
            url = url.replace("https://www.youtube.com/watch?v=", '')
            request = youtube.commentThreads().list(part="snippet,replies",videoId=url)
            response = request.execute()
            covid_comedy_youtube_comments.insert_one(response)
        except:
            continue

 

 if __name__ == "__main__":
    main()

In [88]:
covid_comedy_youtube_comments.count()

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


621

In [96]:
#covid_comedy_youtube_comments.find_one()

### Drafts 

In [97]:

# scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# def main():
#     # Disable OAuthlib's HTTPS verification when running locally.
#     # *DO NOT* leave this option enabled in production.
#     os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

#     api_service_name = "youtube"
#     api_version = "v3"
#     client_secrets_file = "../client_secret_comedy-project-276920.json"

#     # Get credentials and create an API client
#     flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
#         client_secrets_file, scopes)
#     credentials = flow.run_console()
#     youtube = googleapiclient.discovery.build(
#         api_service_name, api_version, credentials=credentials)

#     request = youtube.channels().list(
#         part="snippet,contentDetails,statistics",
#         id="XXXX"
#     )
#     response = request.execute()

#     print(response)

# if __name__ == "__main__":
#     main()

In [98]:
# scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# def main():
#     # Disable OAuthlib's HTTPS verification when running locally.
#     # *DO NOT* leave this option enabled in production.
#     os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

#     api_service_name = "youtube"
#     api_version = "v3"
#     client_secrets_file = "../client_secret_comedy-project-276920.json"

#     # Get credentials and create an API client
#     flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
#         client_secrets_file, scopes)
#     credentials = flow.run_console()
#     youtube = googleapiclient.discovery.build(
#         api_service_name, api_version, credentials=credentials)

#     request = youtube.channels().list(
#         part="snippet,contentDetails,statistics",
#         id="XXXX"
#     )
#     response = request.execute()

#     print(response)

# if __name__ == "__main__":
#     main()